In [1]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Regression

## 数据读取与处理

In [ ]:
from sklearn.datasets import load_boston

data = load_boston()
X = data.data
Y = data.target.reshape((-1,1))

scaler=StandardScaler()
X_sca=scaler.fit_transform(X)

X_train,X_val,Y_train,Y_val=train_test_split(X_sca,Y,test_size=0.15)

print(X_train.shape, Y_train.shape,X_val.shape,Y_val.shape)

## 网络结构设计

In [ ]:
# 神经网络编程有一点与SKlearn不一样，就是需要提前设计好网络的结构，每一层的参数都跟其输入有关
# 输入层单元数为13(数据特征)，设计隐藏层单元数为5，输出单元数为1(回归任务)

unit_I = X_train.shape[1]    # 输入层的单元数，与特征数相等
unit_h1 = 5    # 第一层隐藏层的单元数
unit_h2 = 3
unit_O = 1    # 输出层单元数

## 搭建网络

In [2]:
# 为了便于添加层数，将其封装起来


def add_layer(inputs, units_I, units_O, act_func=None, layer_name=None):
    '''
    inputs: 当前层的输入
    units_I: 当前层的单元数，即输入数据的列数
    units_O: 后一层的单元数，即输出数据的列数
    '''
    w = tf.Variable(tf.random_normal(shape=[units_I, units_O]))
    b = tf.Variable(tf.random_normal(shape=[units_O]))
    z=tf.add(tf.matmul(inputs, w), b)

    if act_func:
        out = act_func(z)    # 激活
    else:
        out = z

    return out

In [ ]:
# 输入必须是可由用户指定的，所以设为placeholder
X = tf.placeholder(tf.float32, shape=[None, unit_I])    # 数据的样本数不指定，只指定特征数
Y = tf.placeholder(tf.float32, shape=[None, 1])    # 目标值为列向量

# 网络结构子图
with tf.name_scope('DNN'):
    a1 = add_layer(X, unit_I, unit_h1, act_func=tf.nn.relu)
    a2 = add_layer(a1, unit_h1, unit_h2, act_func=tf.nn.relu)
    Y_pred = add_layer(a2, unit_h2, unit_O)

# 损失函数子图
with tf.name_scope('Loss'):
    MSE = tf.losses.mean_squared_error(Y, Y_pred)

# optimization子图
lr = 0.01    # 学习率
with tf.name_scope('Train'):
    opt = tf.train.AdamOptimizer(lr).minimize(MSE)    # Adam优化器

init = tf.global_variables_initializer()    # 所有变量初始化

## 训练网络

In [ ]:
# 计算图已经构建好，开启一个tf会话，需要计算哪个值就run哪个变量即可
with tf.Session() as sess:
    sess.run(init)
    train_iter = 1000

    for i in range(train_iter):
        sess.run(opt, feed_dict={X: X_train, Y: Y_train})
        if i % 100 == 0:
            mse_train = sess.run(MSE, feed_dict={X: X_train, Y: Y_train})
            print('train_iter: {}\tMSE:{}'.format(i, mse_train))

    mse_val = sess.run(MSE, feed_dict={X: X_val, Y: Y_val})

# 另一种计算mse_val方式，因为tf计算出来的pred是一个二维数组，需要squeeze一下
#     pred = np.squeeze(sess.run(Y_pred, feed_dict={X: X_val}))
#     mse_val=np.sum(np.square(np.squeeze(pred)-Y_val))/len(Y_val)

    print('test_MSE: {}'.format(mse_val))

# Binary classification
## 数据读取与处理

In [ ]:
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer()
X = data.data
Y = data.target.reshape((-1,1))

scaler=StandardScaler()
X_sca=scaler.fit_transform(X)

X_train,X_val,Y_train,Y_val=train_test_split(X_sca,Y,test_size=0.15)

print(X_train.shape, Y_train.shape,X_val.shape,Y_val.shape)

## 网络结构设计

In [ ]:
# 输入层单元数为30数据特征)，设计隐藏层单元数为5，输出单元数为1(二分类任务)

unit_I = X_train.shape[1]    # 输入层的单元数，与特征数相等
unit_h1 = 5    # 第一层隐藏层的单元数
unit_h2 = 3
unit_O = 1    # 输出层单元数

## 搭建网络

In [ ]:
# 输入必须是可由用户指定的，所以设为placeholder
X = tf.placeholder(tf.float32, shape=[None, unit_I])    # 数据的样本数不指定，只指定特征数
Y = tf.placeholder(tf.float32, shape=[None, 1])    # 目标值为列向量

# 网络结构子图
with tf.name_scope('DNN'):
    a1 = add_layer(X, unit_I, unit_h1, act_func=tf.nn.relu)
    a2 = add_layer(a1, unit_h1, unit_h2, act_func=tf.nn.relu)
    Y_pred = add_layer(a2, unit_h2, unit_O, act_func=tf.nn.sigmoid)

# 损失函数子图
with tf.name_scope('Loss'):
    log_loss = tf.losses.log_loss(Y, Y_pred)

# optimization子图
lr = 0.01    # 学习率
with tf.name_scope('Train'):
    opt = tf.train.AdamOptimizer(lr).minimize(log_loss)    # Adam优化器

init = tf.global_variables_initializer()    # 所有变量初始化

## 训练网络

In [ ]:
# 计算图已经构建好，开启一个tf会话，需要计算哪个值就run哪个变量即可
with tf.Session() as sess:
    sess.run(init)
    train_iter = 1000

    for i in range(train_iter):
        sess.run(opt, feed_dict={X: X_train, Y: Y_train})
        if i % 100 == 0:
            loss_train = sess.run(log_loss, feed_dict={X: X_train, Y: Y_train})
            print('train_iter: {}\tlog_loss:{}'.format(i, loss_train))

    loss_val = sess.run(log_loss, feed_dict={X: X_val, Y: Y_val})
    pred=np.squeeze(sess.run(Y_pred, feed_dict={X: X_val}))
    pred=np.where(pred>=0.5,1,0)
    
    print('test_log_loss: {}\tacc:{}'.format(loss_val,np.sum(pred==np.squeeze(Y_val))/len(Y_val)))

# Multi classification

## 数据读取与处理

In [3]:
from sklearn.datasets import load_digits

data = load_digits()
X = data.data
Y = data.target.reshape((-1,1))

scaler=StandardScaler()
X_sca=scaler.fit_transform(X)

X_train,X_val,Y_train,Y_val=train_test_split(X_sca,Y,test_size=0.15)

print(X_train.shape, Y_train.shape,X_val.shape,Y_val.shape)

(1527, 64) (1527, 1) (270, 64) (270, 1)


## 网络结构设计

In [5]:
# 输入层单元数为64(数据特征)，设计隐藏层单元数为5，输出单元数为10(多分类任务)

unit_I = X_train.shape[1]    # 输入层的单元数，与特征数相等
unit_h1 = 5    # 第一层隐藏层的单元数
unit_h2 = 3
unit_O = 10    # 输出层单元数

## 搭建网络

In [7]:
# 输入必须是可由用户指定的，所以设为placeholder
X = tf.placeholder(tf.float32, shape=[None, unit_I])    # 数据的样本数不指定，只指定特征数
Y = tf.placeholder(tf.int32, shape=[None, 1])    # 目标值为列向量

# 网络结构子图
with tf.name_scope('DNN'):
    a1 = add_layer(X, unit_I, unit_h1, act_func=tf.nn.relu)
    a2 = add_layer(a1, unit_h1, unit_h2, act_func=tf.nn.relu)
    Y_pred = add_layer(a2, unit_h2, unit_O, act_func=tf.nn.softmax)

# 损失函数子图
with tf.name_scope('Loss'):
    # 计算一维向量与onehot向量之间的损失
    cross_entropy = tf.losses.sparse_softmax_cross_entropy(labels=Y, logits=Y_pred)

# optimization子图
lr = 0.01    # 学习率
with tf.name_scope('Train'):
    opt = tf.train.AdamOptimizer(lr).minimize(cross_entropy)    # Adam优化器

init = tf.global_variables_initializer()    # 所有变量初始化

## 训练网络

In [21]:
# 计算图已经构建好，开启一个tf会话，需要计算哪个值就run哪个变量即可
with tf.Session() as sess:
    sess.run(init)
    train_iter = 1000

    for i in range(train_iter):
        sess.run(opt, feed_dict={X: X_train, Y: Y_train})
        if i % 100 == 0:
            loss_train = sess.run(cross_entropy, feed_dict={
                                  X: X_train, Y: Y_train})
            print('train_iter: {}\tlog_loss:{}'.format(i, loss_train))

    pred = sess.run(Y_pred, feed_dict={X: X_val, Y: Y_val})
    loss_val = sess.run(cross_entropy, feed_dict={X: X_val, Y: Y_val})
    acc = np.sum(np.squeeze(Y_val) == np.argmax(pred, axis=1))/len(Y_val)

    print('test_log_loss: {}\tacc:{}'.format(loss_val, acc))

train_iter: 0	log_loss:2.3281965255737305
train_iter: 100	log_loss:1.9704219102859497
train_iter: 200	log_loss:1.7735627889633179
train_iter: 300	log_loss:1.7169830799102783
train_iter: 400	log_loss:1.6748764514923096
train_iter: 500	log_loss:1.663661003112793
train_iter: 600	log_loss:1.656128168106079
train_iter: 700	log_loss:1.6525875329971313
train_iter: 800	log_loss:1.635619044303894
train_iter: 900	log_loss:1.6313313245773315
test_log_loss: 1.6515076160430908	acc:0.8148148148148148
